In [7]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Configuration Dash (assets one level up)
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)

app.title = "⚡ Electronic Device Carousel"

# 📦 Sous-catégories
ELECTRONIC_ITEMS = sorted([
    item for item in os.listdir("../assets/object/electronic_device")
    if os.path.isdir(f"../assets/object/electronic_device/{item}")
])

# 📦 Dictionnaire {item: [images]}
electronic_images = {
    item: sorted([
        os.path.abspath(f"../assets/object/electronic_device/{item}/{img}")
        for img in os.listdir(f"../assets/object/electronic_device/{item}")
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for item in ELECTRONIC_ITEMS
}

# 🎨 Layout
app.layout = html.Div([
    html.H2("⚡ Electronic Device Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
    html.Label("Select electronic device:", style={
        "marginBottom": "6px", "fontSize": "15px", "fontStyle": "normal"
    }),
    dcc.Dropdown(
        id='item-dropdown',
        options=[{"label": item.replace("_", " ").capitalize(), "value": item} for item in ELECTRONIC_ITEMS],
        value=ELECTRONIC_ITEMS[0],
        clearable=False,
        style={"width": "300px", "margin": "0 auto"}
    )
], style={"textAlign": "center", "marginBottom": "20px"}),


    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Affiche carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value')
)
def update_carousel(item):
    images = electronic_images.get(item, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))
    dash_path = f"/assets/{relative_path}"

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "20px", "padding": "6px 12px", "marginRight": "20px"}),

            html.Img(id="carousel-image", src=dash_path,
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),

            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "20px", "padding": "6px 12px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "15px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),

        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "13px", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Lancer
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8806)
